In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [8]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [9]:
token=tokenizer.encode("i am not at my best mood today.",return_tensors='pt')

In [10]:
token

tensor([[  101,   151, 10345, 10497, 10160, 11153, 11146, 69216, 13980,   119,
           102]])

In [12]:
result=model(token)

In [13]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2574,  1.9356,  2.0232, -0.7362, -2.7159]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [15]:
int(torch.argmax(result.logits))

2

In [24]:
r = requests.get('https://www.yelp.com/biz/the-sage-and-drifter-san-francisco?osq=bars')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [25]:
results

[<p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">One of the best bars the city has to offer.  Stepping in the door here is like entering a hidden gem in the heart of the city.  <br/><br/>The first thing that catches your eye is the impeccable attention to their craft.</span></p>,
 <p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">Thank goodness for a neighborhood cocktail place! Came on a Thursday night around 730 and there was about 7 people in the bar. We decided on the weary drifter (5/5) and the hongo picante (4/5). This season's weary drifter included beets which gave it beautiful color and reminded me of a pisco sour variation. Bartender told us they change the weary drifter every 2-3 months. The mushroom infused tequila was so interesting and I was impressed you could actually taste it! Parking is difficult in this area but a great place for night cap. About 10 good craft beers also on tap.</span></p>,

In [27]:
reviews

['One of the best bars the city has to offer.  Stepping in the door here is like entering a hidden gem in the heart of the city.  The first thing that catches your eye is the impeccable attention to their craft.',
 "Thank goodness for a neighborhood cocktail place! Came on a Thursday night around 730 and there was about 7 people in the bar. We decided on the weary drifter (5/5) and the hongo picante (4/5). This season's weary drifter included beets which gave it beautiful color and reminded me of a pisco sour variation. Bartender told us they change the weary drifter every 2-3 months. The mushroom infused tequila was so interesting and I was impressed you could actually taste it! Parking is difficult in this area but a great place for night cap. About 10 good craft beers also on tap.",
 "After having a quick burger in Walnut Creek, my friend and I decided it'll be best to beat the traffic and head back to the city. He looked on Yelp to see what was in the area nearby to where we were g

In [29]:
import numpy as np
import pandas as pd

In [30]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [31]:
df['review'].iloc[0]

'One of the best bars the city has to offer.  Stepping in the door here is like entering a hidden gem in the heart of the city.  The first thing that catches your eye is the impeccable attention to their craft.'

In [40]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))

In [43]:
sentiment_score(review=df['review'].iloc[10])

4

In [44]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [45]:
df

,review,sentiment
0,One of the best bars the city has to offer. S...,4
1,Thank goodness for a neighborhood cocktail pla...,4
2,"After having a quick burger in Walnut Creek, m...",4
3,If you live in The Sets...After a looooong day...,3
4,Dope spot with unique well thought out drinks....,4
5,"Very relaxed, easy space. Well executed and th...",4
6,What a fun bar experience! I was meeting a fri...,4
7,"As we enter a new 2023, I began reflecting on ...",3
8,This place is such a hidden gem. I got the Ken...,4
9,Here for the good drinks and good people. They...,4


In [46]:
df['review'].iloc[3]

'If you live in The Sets...After a looooong day of working from home in your PJs, sometimes you want to have a craft cocktail without getting all dolled up to head to The Mission, SoMa, or The Marina.  Well, throw on your puffy jacket and some uggs, and head to The Sage & Drifter! You don\'t even need to comb out the bird\'s nest on your head because no judgement.The Sage & Drifter is welcoming neighborhood bar that\'s low-key. So, come as you are, pull up a seat and get your drank on!I haven\'t been to this spot since it was "The Corner Spot" and it hasn\'t changed much, other than the addiiton of a craft cocktail menu ($15 each): https://tinyurl.com/mr6vvtj4. I want to get my buzz on STAT so opted for Casamigos Blanco, double on the rocks: https://tinyurl.com/5e8d6e4k. It\'s smooth with vanilla and citrus notes. It\'s my go-to if Fortaleza Blanco isn\'t available. Three shot of Casamigos Blanco, Fernet Neat, and a soda was $45 pre-tip. I think it was discounted for HH so very afforda